In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# 1. Charger la base de données
df = pd.read_csv('base_anomalies_6084.csv')

# 2. Sélectionner les colonnes pertinentes
features = ['Statut de salariés', 'Frontalier', 'Rub 6084', '6084Taux']
target = 'anomalie_crds'

# 3. Convertir les variables catégorielles en numériques (Statut de salariés, Frontalier)
label_encoder_statut = LabelEncoder()
label_encoder_frontalier = LabelEncoder()

df['Statut de salariés'] = label_encoder_statut.fit_transform(df['Statut de salariés'])
df['Frontalier'] = label_encoder_frontalier.fit_transform(df['Frontalier'])

# 4. Préparer les données pour le modèle
X = df[features]
y = df[target]

# 5. Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Créer et entraîner le modèle RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 7. Faire des prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# 8. Évaluer la performance du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy * 100))

# Rapport de classification pour plus de détails
print("Rapport de classification :\n", classification_report(y_test, y_pred))


Précision du modèle sur les données de test : 100.00%
Rapport de classification :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     87439
           1       1.00      1.00      1.00      9696

    accuracy                           1.00     97135
   macro avg       1.00      1.00      1.00     97135
weighted avg       1.00      1.00      1.00     97135



In [23]:
import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, 'random_forest_model_6084.pkl')


['random_forest_model_6084.pkl']

In [17]:
import pandas as pd
import numpy as np

# 1. Charger la base de données existante
df = pd.read_csv('updated_file_with_region.csv')

# 2. Définir le taux d'anomalies (par exemple 10 % des lignes)
taux_anomalie = 0.10  # 10 % d'anomalies

# 3. Sélectionner aléatoirement un certain pourcentage de lignes pour introduire des anomalies
n_lignes = df.shape[0]
n_anomalies = int(taux_anomalie * n_lignes)

# 4. Sélectionner des indices aléatoires distincts pour les anomalies
indices_anomalies_crds = np.random.choice(df.index, size=n_anomalies, replace=False)

# 5. Ajouter une colonne pour marquer les anomalies
df['anomalie_crds'] = 0  # Initialement, aucune anomalie

# 6. Fonction pour générer un taux aléatoire erroné, arrondi à 2 décimales
def generer_taux_errone(eliminer_valeur=None, valeur_max=10):
    taux_errone = round(np.random.uniform(0, valeur_max), 2)  # Choisissez une plage appropriée et arrondissez
    while eliminer_valeur is not None and taux_errone == eliminer_valeur:
        taux_errone = round(np.random.uniform(0, valeur_max), 2)  # Générer jusqu'à ce ne soit pas la valeur à éliminer
    return taux_errone

# 7. Fonction pour introduire des anomalies dans la cotisation CRDS
def introduire_anomalie_crds(row):
    frontalier = row['Frontalier']
    statut = row['Statut de salariés']

    # Conditions d'anomalie
    if frontalier == 'Frontaliers' and statut == 'Stagiaire':
        # Taux correct est 0%
        row['6084Taux'] = generer_taux_errone(eliminer_valeur=0)  # Taux erroné
        row['anomalie_crds'] = 1
    elif frontalier == 'Non' and statut == 'Stagiaire':
        # Taux correct est 0%
        row['6084Taux'] = generer_taux_errone(eliminer_valeur=0)  # Taux erroné
        row['anomalie_crds'] = 1
    elif frontalier == 'Frontaliers' and statut != 'Stagiaire':
        # Taux correct est 0%
        row['6084Taux'] = generer_taux_errone(eliminer_valeur=0)  # Taux erroné
        row['anomalie_crds'] = 1
    elif frontalier == 'Non' and statut != 'Stagiaire':
        # Taux correct est 0.5%
        row['6084Taux'] = generer_taux_errone(eliminer_valeur=0.5)  # Taux erroné
        row['anomalie_crds'] = 1

    return row

# 8. Appliquer la fonction d'anomalie uniquement sur les lignes sélectionnées
df.loc[indices_anomalies_crds, :] = df.loc[indices_anomalies_crds, :].apply(introduire_anomalie_crds, axis=1)

# 9. Sauvegarder la nouvelle base de données avec anomalies
df.to_csv('base_anomalies_crds.csv', index=False)

# Optionnel : Afficher un résumé des anomalies introduites
#total_anomalies_crds = df['anomalie_crds'].sum()
#print(f"Total des anomalies introduites pour la cotisation CRDS : {total_anomalies_crds} sur {n_lignes} lignes ({(total_anomalies_crds/n_lignes)*100:.2f}%)")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 16: invalid continuation byte

In [25]:
import pandas as pd
import joblib

# 1. Charger le modèle entraîné
rf_classifier_loaded = joblib.load('random_forest_model_6084.pkl')

# 2. Charger la nouvelle base de données (sans la colonne 'anomalie_csg')
new_df = pd.read_csv('updated_file_with_region.csv')

# 3. Sélectionner les mêmes colonnes que celles utilisées pour l'entraînement
features_new = new_df[['Statut de salariés', 'Frontalier', 'Rub 6084', '6084Taux']]

# 4. Encodage des variables catégorielles (Statut de salariés, Frontalier)
# Vous devez utiliser les mêmes encodeurs que ceux utilisés lors de l'entraînement
label_encoder_statut = LabelEncoder()
label_encoder_frontalier = LabelEncoder()

# Encodage des variables catégorielles dans la nouvelle base
features_new['Statut de salariés'] = label_encoder_statut.fit_transform(features_new['Statut de salariés'])
features_new['Frontalier'] = label_encoder_frontalier.fit_transform(features_new['Frontalier'])

# 5. Vérifier que le nombre de colonnes correspond bien à celles utilisées pour l'entraînement
# Si certaines colonnes manquent (cas rare), on les ajoute avec des valeurs par défaut
missing_cols = set(X.columns) - set(features_new.columns)  # 'X' ici représente les colonnes d'entraînement
for c in missing_cols:
    features_new[c] = 0  # Ajouter des colonnes manquantes avec une valeur par défaut

# 6. Réarranger les colonnes dans le même ordre que lors de l'entraînement
features_new = features_new[X.columns]

# 7. Utiliser le modèle pour prédire les anomalies sur la nouvelle base
predictions = rf_classifier_loaded.predict(features_new)

# 8. Ajouter la colonne 'anomalie_predite' pour indiquer les prédictions faites par le modèle
new_df['anomalie_predite'] = predictions

# 9. Sauvegarder la nouvelle base de données avec les prédictions
new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print("Prédictions ajoutées à la nouvelle base de données.")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 16: invalid continuation byte

test

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Charger la base de données
df = pd.read_csv('base_anomalies_6084.csv')

# 2. Sélectionner les colonnes pertinentes
features = ['Statut de salariés', 'Frontalier', 'Rub 6084', '6084Taux']
target = 'anomalie_crds'

# 3. Convertir les variables catégorielles en numériques (Statut de salariés, Frontalier)
label_encoder_statut = LabelEncoder()
df['Statut de salariés'] = label_encoder_statut.fit_transform(df['Statut de salariés'])

label_encoder_frontalier = LabelEncoder()
df['Frontalier'] = label_encoder_frontalier.fit_transform(df['Frontalier'])

# Sauvegarder les encodeurs
joblib.dump(label_encoder_statut, 'label_encoder_statut.pkl')
joblib.dump(label_encoder_frontalier, 'label_encoder_frontalier.pkl')

# 4. Préparer les données pour le modèle
X = df[features]
y = df[target]

# 5. Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 6. Créer et entraîner le modèle RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# 7. Faire des prédictions sur les données de test
y_pred = rf_classifier.predict(X_test)

# 8. Évaluer la performance du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision du modèle sur les données de test : {:.2f}%".format(accuracy * 100))

# Rapport de classification pour plus de détails
print("Rapport de classification :\n", classification_report(y_test, y_pred))


Précision du modèle sur les données de test : 100.00%
Rapport de classification :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     87439
           1       1.00      1.00      1.00      9696

    accuracy                           1.00     97135
   macro avg       1.00      1.00      1.00     97135
weighted avg       1.00      1.00      1.00     97135



In [32]:
import joblib

# Exporter le modèle Random Forest entraîné
joblib.dump(rf_classifier, 'random_forest_model_6084_new.pkl')

['random_forest_model_6084_new.pkl']

In [ ]:
import pandas as pd
import joblib

# 1. Charger le modèle entraîné
rf_classifier_loaded = joblib.load('random_forest_model_6084_new.pkl')

# 2. Charger la nouvelle base de données (sans la colonne 'anomalie_csg')
new_df = pd.read_csv('new_updated_file 2.csv')

# 3. Charger les encodeurs
label_encoder_statut = joblib.load('label_encoder_statut.pkl')
label_encoder_frontalier = joblib.load('label_encoder_frontalier.pkl')

# 4. Sélectionner les mêmes colonnes que celles utilisées pour l'entraînement
features_new = new_df[['Statut de salariés', 'Frontalier', 'Rub 6084', '6084Taux']]

# 5. Encodage des variables catégorielles (Statut de salariés, Frontalier)
features_new['Statut de salariés'] = label_encoder_statut.transform(features_new['Statut de salariés'])
features_new['Frontalier'] = label_encoder_frontalier.transform(features_new['Frontalier'])

# 6. Vérifier que le nombre de colonnes correspond bien à celles utilisées pour l'entraînement
# Si certaines colonnes manquent (cas rare), on les ajoute avec des valeurs par défaut
missing_cols = set(X.columns) - set(features_new.columns)  # 'X' ici représente les colonnes d'entraînement
for c in missing_cols:
    features_new[c] = 0  # Ajouter des colonnes manquantes avec une valeur par défaut

# 7. Réarranger les colonnes dans le même ordre que lors de l'entraînement
features_new = features_new[X.columns]

# 8. Utiliser le modèle pour prédire les anomalies sur la nouvelle base
predictions = rf_classifier_loaded.predict(features_new)

# 9. Ajouter la colonne 'anomalie_predite' pour indiquer les prédictions faites par le modèle
new_df['anomalie_predite'] = predictions

# 10. Sauvegarder la nouvelle base de données avec les prédictions
new_df.to_csv('nouvelle_base_avec_predictions.csv', index=False)

print("Prédictions ajoutées à la nouvelle base de données.")

